Deploy the model Endpoint

In [ ]:
import sagemaker
from sagemaker.sklearn.model import SKLearnModel
from sagemaker import get_execution_role

# 1. Setup
sagemaker_session = sagemaker.Session()
role = get_execution_role()

# S3 location of your packaged model tarball

model_data_s3_path = 's3://ecomrecdata/model_artifacts/model.tar.gz'

# The version of the Scikit-learn container to use
# Check AWS documentation for the latest versions
framework_version = '1.2-1'
py_version = 'py3'

# 2. Create a SageMaker Model

model = SKLearnModel(
    model_data=model_data_s3_path,
    role=role,
    entry_point='inference.py',
    framework_version=framework_version,
    py_version=py_version,
    sagemaker_session=sagemaker_session
)

# 3. Deploy the Model to a Real-time Endpoint
predictor = model.deploy(
    initial_instance_count=1,
    instance_type='ml.t2.medium',
    endpoint_name='hybrid-recommender-endpoint'
)

print(f"Deployment complete. Endpoint Name: {predictor.endpoint_name}")

# 4. Example Inference Call
test_payload = {
    "user_id": 483717,
    "alpha": 0.7,
    "top_n": 5
}

# The predictor will automatically call input_fn, predict_fn, and output_fn
response = predictor.predict(test_payload, initial_args={'ContentType': 'application/json', 'Accept': 'application/json'})

print("\n--- Inference Response ---")
print(response)

# 5. Clean up (Important to avoid unnecessary charges)
predictor.delete_endpoint()